In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import pickle

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [15]:
# Load saved keys from key file
keys = pickle.load(open('api_key.p', 'rb'))

In [18]:
auth = tw.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

In [30]:
# See available regions
# api.trends_available() # worldwide has woeid of 1

# Get worldwide trending topics and number of Tweets for each
trends1 = api.trends_place(1)
trends = [(trend['name'], trend['tweet_volume']) for trend in trends1[0]['trends']]

# Create pandas df
trends_df = pd.DataFrame(trends, columns=['Topic', 'Count'])
trends_df.head()

,Topic,Count
0,#WHUARS,69165.0
1,#ImpeachmentHearings,150606.0
2,#bbcqt,44633.0
3,West Ham,55354.0
4,#IMedici,29731.0


In [33]:
# Download tweets based on trending topic
search_term = trends[0][0] + ' -filter:retweets'

tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2019-12-09').items(10)

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:5]

['Ironic how Xhaka spent the whole game ducking out of tackles to protect his face the gets smashed in the face with… https://t.co/0DNSlmspW2',
 'Funny what one game, 5 mins of football can do- feeling so happy like the players now! They are smiling again. Feel… https://t.co/XAW1AnGJmX',
 'If we assume Pellegrini is out, Eddie Howe, Moyes and Benitez would all be in the running. I don’t like the way the… https://t.co/u9gawpiAUg',
 "@karren_brady @davidgold @WestHam it's time to act before it gets to late.\n\nThis nonsense needs to stop now\n\n#WHUFC… https://t.co/8SoSdxyA6m",
 'On the brink... but Manuel Pellegrini backs himself to solve West Ham crisis amid fan fury\n\n#WHUARS  https://t.co/MpBFyltXUd']